# Tool

## Tools 是什麼？

工具就是 Agent 執行外部動作的能力，例如：

- 查詢資料庫
- 呼叫 API
- 搜尋資料
- 計算公式
- 存取檔案

## Tool 的特性
在 LangGraph / LangChain 中：

> 工具本質上就是「有明確輸入與輸出的 Python 函式」

關鍵特性：

1. 輸入有型別（type hints）
2. 輸出可序列化（通常是 str / dict / list）
3. 有清楚的語意描述（docstring）

這讓 LLM 可以：

- 理解工具能做什麼
- 自動決定是否呼叫
- 正確產生參數

## Tool 範例
### 1. 最簡工具範例（純 Python）
這個版本節點中可以直接呼叫使用，但 LLM 無法自動選擇它，因為它沒有 metadata。

In [ ]:
def search_tool(query: str) -> str:
    return f"搜尋結果：{query} 的相關資料…"

### 2. 使用 @tool 裝飾器
LangChain 提供 @tool 裝飾器，用來把普通函式變成「LLM 可理解的工具」。

這個裝飾器會幫你完成：
1. 擷取函式名稱（tool name）
2. 解析輸入參數 schema（JSON Schema）
3. 讀取 docstring 當作工具說明

In [ ]:
from langchain.tools import tool

@tool
def search_tool(query: str) -> str:
    """搜尋網路上與 query 相關的資訊"""
    return f"搜尋結果：{query} 的相關資料…"

## 在 Node 中使用工具
假設我們加一個搜尋工具（假裝它會搜尋網路）：

In [ ]:
def research_node(state):
    topic = state["topic"]

    # 使用上面定義好的 tool
    results = search_tool(topic)
    return {"research": results}

這樣 Node 就能使用工具。

# Conditional Edges


## Conditional Edges 是什麼？

Conditional Edge 用來根據節點執行後的結果，決定下一步要走向哪個節點。
> Conditional Edge = 控制 Agent 決策流程的關鍵機制

## Conditional Edge 的用途

常見使用情境：

- 根據 LLM 輸出的內容決定是否呼叫工具
- 判斷任務是否完成，決定要不要繼續循環
- 依不同分類結果走向不同處理流程
- 發生錯誤時導向修正或結束節點

## Conditional Edge 的特性
在 LangGraph 中：
> 條件邊本質上就是「一個根據 state 回傳路徑名稱的函式」

關鍵特性：

1. 輸入通常是目前的 state
2. 輸出是一個 分支 key（字串）
3. 每個 key 對應一條邊與下一個節點

LLM 不會直接選邊，而是：
- 節點先更新 state
- 條件函式根據 state 做判斷
- Graph 根據回傳值決定下一步

## 設計 Conditional Edge 的建議

1. 回傳值要簡單、可預期
    - 使用固定字串（如 "yes", "no", "retry"）
2. 判斷邏輯不要太複雜
    - 複雜推理交給 LLM
3. 避免把業務邏輯塞進條件函式
    - Conditional Edge 只做「結果判斷」

## Conditional Edge 範例
### 簡易 Conditional Edge範例
假設 state 中有一個欄位 is_done，表示任務是否完成。

In [ ]:
def route_by_status(state: dict) -> str:
    if state["is_done"]:
        return "end"
    else:
        return "continue"

### 將 Conditional Edge 加入 Graph
使用 add_conditional_edges 將條件函式掛到節點後。

In [ ]:
graph.add_conditional_edges(
    "check_status",
    route_by_status,
    {
        "continue": "agent_node",
        "end": END,
    }
)

執行邏輯為：

1. 執行 check_status 節點

2. 呼叫 route_by_status(state)

3. 根據回傳值：

    - "continue" → 前往 agent_node
    - "end" → 結束流程

> add_conditional_edges 一定要第三個參數（mapping）

## Command 與 goto
除了回傳字串決定分支外，LangGraph 也支援使用 Command 物件 來更明確地控制流程跳轉。
### Command / goto 是什麼？
> Command 是一個「顯式流程控制指令」

它可以告訴 Graph：
- 下一步要去哪個節點（goto）
- 同時要對 state 做哪些更新（update）

### Command goto 範例

In [ ]:
from langgraph.types import Command
from langgraph.graph import END

def route_decision(state: dict):
    if state["score"] > 0.7:
        return Command(goto="finalize")
    else:
        return Command(goto="write")

- 不需要再定義分支 key
- 直接指定要跳轉的節點名稱
- Graph 會立刻依指令改變流程



### Command 同時更新 state

In [ ]:
def route_decision(state: dict):
    score = state["score"]
    if state["score"] > 0.7:
        return Command(
            goto="finalize",
            update={"score": score}
        )
    else:
        return Command(
            goto="write",
            update={"retry_count": state.get("retry_count", 0) + 1}
        )


### Command goto 缺點：
流程圖不完整/不直觀：

In [ ]:
graph.add_edge("write", "review")
graph.add_edge("finalize", END)

# 實際運行的完整流程:
# write → review → route_decision → write (循環)
#                                 → finalize → END

## Conditional Edge 實作：審核 → 不滿意就重寫

我們將建立以下流程：

In [ ]:
write → review → (不滿意?) → write → review → … → finalize

### Step 0：定義初始 state

In [ ]:
from typing import TypedDict

class WritingState(TypedDict):
    topic: str
    article: str
    score: float

### Step 1：寫文章 Node

In [ ]:
def write_node(state: WritingState) -> WritingState:
    topic = state["topic"]
    prompt = f"請寫一段 80~120 字的介紹:{topic}"
    result = llm.invoke(prompt).content
    print("[write_node] 生成文章")
    return {"article": result}

### Step 2：審核 Node

In [ ]:
def review_node(state: WritingState) -> WritingState:
    article = state["article"]
    prompt = f"請評估以下文章品質，給 0~1 分數，只回傳數字：{article}"
    score = float(llm.invoke(prompt).content)
    
    print(f"[review_node] score: {score}")

    if score >= 0.7:
        print("[review_node] 分數達標,前往 finalize")
        return Command(update={"score": score}, goto="finalize")
    print(f"[review_node] 分數不足, 重寫")
    return Command(update={"score": score}, goto="write")

### Step 3：finalizer Node

In [ ]:
def finalize_node(state: WritingState):
    print(f"[finalize_node] 最終文章: {state['article']}")
    return {"final": state["article"]}

### 把它們加進 Graph

In [ ]:
from langgraph.graph import StateGraph, END

graph = StateGraph(WritingState)

graph.add_node("write", write_node)
graph.add_node("review", review_node)
graph.add_node("finalize", finalize_node)

graph.set_entry_point("write")
graph.add_edge("write", "review")
# review 內部用 Command 跳到 write 或 finalize
graph.add_edge("finalize", END)

app = graph.compile()

### 這個範例中，有個隱含的風險
現在這個版本：

In [ ]:
if state["score"] < 0.7:
    
    return Command(goto="write")

-> 如果怎麼寫都寫不到 0.7，會無限 loop

## 避免無限 Loop 的實務做法
### 加入 retry / loop counter

In [ ]:
class WritingState(TypedDict):
    topic: str
    article: str
    score: float
    retry_count: int

MAX_RETRY = 3

In [ ]:
def review_node(state: WritingState):
    article = state["article"]
    prompt = f"請評估以下文章品質,給 0~1 分數,只回傳數字:\n{article}"
    score = float(llm.invoke(prompt).content)
    
    print(f"[review_node] score: {score}")
    
    retry = state.get("retry_count", 0)
    
    if score >= 0.7:
        print("[review_node] 分數達標,前往 finalize")
        return Command(update={"score": score}, goto="finalize")
    
    if retry >= MAX_RETRY:
        print(f"[review_node] 已達最大嘗試次數,強制結束")
        return Command(update={"score": score}, goto="finalize")
    
    print(f"[review_node] 分數不足,第 {retry + 1} 次重寫")
    return Command(
        update={"score": score, "retry_count": retry + 1},
        goto="write"
    )

### 完整程式：

In [2]:
from typing import TypedDict
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, END
from langgraph.types import Command 
import os

# -------------------------
# LLM 設定(GPT-4o-mini)
# -------------------------
os.environ["OPENAI_API_KEY"] = "sk-proj-XXXXXX"
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.7
)

# -------------------------
# Step 0：定義初始 state
# -------------------------
class WritingState(TypedDict):
    topic: str
    article: str
    score: float
    retry_count: int

MAX_RETRY = 3

# -------------------------
# Step 1：寫文章 Node
# -------------------------
def write_node(state: WritingState) -> WritingState:
    topic = state["topic"]
    prompt = f"請寫一段 80~120 字的介紹:{topic}"
    result = llm.invoke(prompt).content
    print("[write_node] 生成文章")
    return {"article": result}

# -------------------------
# Step 2：審核 Node
# -------------------------
def review_node(state: WritingState):
    article = state["article"]
    prompt = f"請評估以下文章品質,給 0~1 分數,只回傳數字:\n{article}"
    score = float(llm.invoke(prompt).content)

    print(f"[review_node] score: {score}")
    
    retry = state.get("retry_count", 0)
    
    if score >= 0.7:
        print("[review_node] 分數達標,前往 finalize")
        return Command(update={"score": score}, goto="finalize")
    
    if retry >= MAX_RETRY:
        print(f"[review_node] 已達最大嘗試次數,強制結束")
        return Command(update={"score": score}, goto="finalize")
    
    print(f"[review_node] 分數不足,第 {retry + 1} 次重寫")
    return Command(
        update={"score": score, "retry_count": retry + 1},
        goto="write"
    )

# -------------------------
# Step 3：finalizer Node
# -------------------------
def finalize_node(state: WritingState):
    print("\n[finalize_node] 最終文章:\n")
    print(state["article"])
    print(f"\n最終分數: {state.get('score', 0)}")
    print(f"重試次數: {state.get('retry_count', 0)}")
    return state

# -------------------------
# Graph 建立
# -------------------------
graph = StateGraph(WritingState)

# 添加節點
graph.add_node("write", write_node)
graph.add_node("review", review_node)
graph.add_node("finalize", finalize_node)

# 設定入口
graph.set_entry_point("write")

# 添加邊
graph.set_entry_point("write")
graph.add_edge("write", "review")
# review 內部用 Command 跳到 write 或 finalize
graph.add_edge("finalize", END)

app = graph.compile()

# -------------------------
# 執行
# -------------------------
result = app.invoke({
    "topic": "人工智慧在教育領域的應用",
    "retry_count": 0,
    "article": "",
    "score": 0.0
})
    
print("\n=== Graph 執行完成 ===")


[write_node] 生成文章
[review_node] score: 1.0
[review_node] 分數不足,第 1 次重寫
[write_node] 生成文章
[review_node] score: 1.0
[review_node] 分數不足,第 2 次重寫
[write_node] 生成文章
[review_node] score: 1.0
[review_node] 分數不足,第 3 次重寫
[write_node] 生成文章
[review_node] score: 1.0
[review_node] 已達最大嘗試次數,強制結束

[finalize_node] 最終文章:

人工智慧在教育領域的應用越來越廣泛，為學習者和教學者提供了全新的工具與方法。透過智能教學系統，學生可以根據個人學習進度獲得量身定制的課程，促進自主學習。此外，AI還能分析學生的學習數據，及時識別學習困難，並提供針對性的輔導方案。教師則可利用AI進行課堂管理、評分自動化及資源推薦，從而提高教學效率。整體而言，人工智慧的介入不僅提升了學習效果，也為教育公平和資源分配提供了新的可能性。

最終分數: 1.0
重試次數: 3

=== Graph 執行完成 ===
